In [1]:
from scipy.optimize import fmin_bfgs, fmin_l_bfgs_b
import csv
import numpy as np
import math
from collections import defaultdict
import csv
import scipy as sc

In [2]:
data = []
with open('100 core subset filtered.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append((row['uid'], row['pid'], float(row['rating']), row['time']))

In [3]:
data[0], len(data)

(('A1YXHALQFWN45C', '0783885008', 4.0, '1288310400'), 67046)

In [4]:
users = {}
products = {}
us = set()
ps = set()
i = 0
j = 0
for d in data:
    if d[0] not in us:
        users[d[0]] = i
        i+=1
        us.add(d[0])
    if d[1] not in ps:
        products[d[1]] = j
        j+=1
        ps.add(d[1])

In [5]:
itemsuserrated = defaultdict(lambda: [])
for d in data:
    itemsuserrated[d[0]].append(d[1])
len(itemsuserrated)

357

In [71]:
# # datag = list(set(datag))
# # np.random.shuffle(datag)
# dtest = []
# dt = set()
# dv = set()
# dvalid= [] #to choose best regualrization parameter and dimninsionality of factor vectors 
# dtestusers = defaultdict(int)
# dvalidusers = defaultdict(int)
# for d in data:
#     cr = len(itemsuserrated[d[0]])
#     vc = int((cr * 20)/float(100))
#     if dtestusers[d[0]]< vc:
#         dtestusers[d[0]] +=1
#         dt.add(d)
#         dtest.append(d)
#     if d not in dt and dvalidusers[d[0]]<vc:
#         dvalidusers[d[0]]+=1
#         dv.add(d)
#         dvalid.append(d)

# dtrain = []
# for d in data:
#     if d not in dt and d not in dv:
#         dtrain.append(d)
# dtrain = list(dtrain)
# dtest = list(dtest)

In [99]:
x60 = int((len(data) * 60) / float(100))
x20 = int((len(data) * 20) / float(100))
dtrain = data[:x60]
dvalid = data[x60: x60+ x20]
end =  x60+ x20
dtest = data[end:]
print len(dtrain), len(dvalid), len(dtest), len(data)
print len(dtrain) + len(dvalid) + len(dtest) == len(data)

40227 13409 13410 67046
True


In [100]:
dtrain[0]

('A1YXHALQFWN45C', '0783885008', 4.0, '1288310400')

In [101]:
print len(users), len(products)
print len(dtrain), len(dtest), len(dvalid), len(data)
print len(dtrain)+ len(dtest)+ len(dvalid) == len(data)

357 50123
40227 13410 13409 67046
True


In [102]:
dtrain[0][0]

'A1YXHALQFWN45C'

In [103]:
us = set()
for d in data:
    us.add(d[0])
ps = set()
for d in data:
    ps.add(d[1])

In [104]:
uintrain = set()
pintrain = set()
gintrain = set()
for d in dtrain:
    pintrain.add(d[1])
    uintrain.add(d[0])

In [105]:
def compute_cost(initial,lam,k, data):
    alpha = initial[0]
    betau = initial[1:len(users) + 1]
    end = len(users)+1
    betai = initial[end:end + len(products)]
    end = end + len(products)
    gs = np.array(initial[end:])
    gr = gs.reshape(len(users)+len(products),k)
    gammu = gr[:len(users),:]
    gammi = gr[len(users):,:]
    J = 0
    for d in data:
        J+= (alpha + betau[users[d[0]]] + betai[products[d[1]]] + (gammu[users[d[0]]].T.dot(gammi[products[d[1]]])) - d[2]) ** 2
    for u in us:
        J += lam * (np.square(betau[users[u]])+ np.sum(np.square(gammu[users[u]])))
    for p in ps:
        J+= lam * ( np.square(betai[products[p]]) + np.sum(np.square(gammi[products[p]])))
    g = compute_gradient(alpha, betau, betai,gammu, gammi,lam,k, data)
#     print J
    return J,g.flatten()

In [106]:
def compute_gradient(alpha,betau, betai,gammu, gammi,lam,k, data):
    ga = 0
    gbu = np.zeros(len(users))
    gbi = np.zeros(len(products))
    ggu = np.zeros((len(users),k))
    ggi = np.zeros((len(products),k))
    for d in data:
        term = alpha + betau[users[d[0]]] + betai[products[d[1]]] + (gammu[users[d[0]]].T.dot(gammi[products[d[1]]])) - d[2]
        ga+= 2 * term
        gbu[users[d[0]]] += 2 * term
        gbi[products[d[1]]] += 2 * term
        ggu[users[d[0]]] += 2 * gammi[products[d[1]]] * term
        ggi[products[d[1]]] += 2 * gammu[users[d[0]]] * term
    for u in us:
        gbu[users[u]] += 2 * lam * betau[users[u]]
        ggu[users[u]] += 2 * lam * gammu[users[u]]
    for p in ps:
        gbi[products[p]] += 2 * lam * betai[products[p]]
        ggi[products[p]] += 2 * lam * gammi[products[p]]
    return np.array([ga] + list(gbu) + list(gbi) + list(ggu.flatten())+ list(ggi.flatten()))

In [107]:
def cost(initial,k, data):
    alpha = initial[0]
    betau = initial[1:len(users) + 1]
    end = len(users)+1
    betai = initial[end:end + len(products)]
    end = end + len(products)
    gs = np.array(initial[end:])
    gr = gs.reshape(len(users)+len(products),k)
    gammu = gr[:len(users),:]
    gammi = gr[len(users):,:]
    J = 0
    for d in data:
        J+= (alpha + betau[users[d[0]]] + betai[products[d[1]]] + (gammu[users[d[0]]].T.dot(gammi[products[d[1]]])) - d[2]) ** 2

    return J

In [108]:
def getPredictions(params,k, data):
    alphao = params[0]
    betauo = params[1: len(users)+1]
    end = len(users)+1
    betaio = params[end: end+len(products)]
    end = end+len(products)
    gs = np.array(params[end:])
    gr = gs.reshape(len(users)+len(products),k)
    gammuo = gr[:len(users),:]
    gammio = gr[len(users):,:]
    predictions = []
    for d in data:
        predictions.append(alphao + betauo[users[d[0]]] + betaio[products[d[1]]]+ (gammuo[users[d[0]]].T.dot(gammio[products[d[1]]])))
    return predictions

In [109]:
def mse(params,k, data):
    s = 0
    predictions = getPredictions(params, k, data)
    for i, d in enumerate(data):
        s+= (predictions[i] - d[2])**2
    return s / float(len(data))

In [110]:
# from sklearn import metrics
# labels = [d[2] for d in dtest]
# predictions = getPredictions(op, 10, dtest)

# metrics.mean_squared_error(labels, predictions), mse(op, 10, dtest)

In [111]:
def mean(data):
    s = 0
    for d in data:
        s+= d[2]
    return  s/ float(len(data))

In [112]:
def variance(data):
    m = mean(data)
    s = 0
    for d in data:
        s += (m - d[2]) ** 2
    return s/float(len(data))

In [113]:
def Rsquare_statistic(params,k, data):
    predictions = getPredictions(params,k, data)
    error = mse(params,k, data)
    var = variance(data)
    return error/float(var)

In [114]:
def rmse(params,k,data):
    predictions = getPredictions(params, k, data)

    s = 0
    for i, d in enumerate(data):
        s+= (predictions[i] - d[2])**2
    return math.sqrt(s / float(len(data)))

In [115]:
# with open('initial parameters.csv', 'wb') as f:
#     writer = csv.DictWriter(f, fieldnames=['value'])
#     writer.writeheader()
#     for i in initialg:
#         writer.writerow({'value': i})

In [116]:
# initialg = []
# with open('initial parameters.csv', 'rb') as csvfile:
#     reader = csv.DictReader(csvfile)
#     for row in reader:
#         initialg.append(float(row['value']))

In [117]:
# class ConvergedException(Exception):
#     pass
# def check(xk):
#     global result
#     global minmse
#     global count
#     global presult
#     global nit 
    
#     nit +=1
# #     print nit

# #     cmse = mse(xk,k, dvalid)
#     cmse = cost(xk, k , dvalid)
#     if cmse > minmse:
#         if nit >= 100:
#             count +=1
#             if count == 5:
#                 print 'here'
#                 result[:] = presult
#                 raise ConvergedException()
#                 return

#     else:
#         minmse = cmse
#         presult[:] = xk
#         count = 0
# #     print cmse, minmse, presult[0]
#     return
    

In [118]:
k = 10
s = 0
for d in data:
    s+= d[2]
alpha = s / float(len(data))
betau = np.random.normal(0,0.1, len(users))
betai = np.random.normal(0,0.1, len(products))
for u in users:
    if u not in uintrain:
        betau[users[u]] = 0
for p in products:
    if p not in pintrain:
        betai[products[p]] = 0
gammu = np.random.normal(0,0.1, (len(users), k)) 
gammi = np.random.normal(0,0.1, (len(products), k)) 
for u in users:
    if u not in uintrain:
        gammu[users[u]] = 0
for p in products:
    if p not in pintrain:
        gammi[products[p]] = 0
initial = [alpha] + list(betau) + list(betai) 
initialg= initial + list(gammu.flatten()) + list(gammi.flatten())

In [119]:
# class ConvergedException(Exception):
#     pass
# def check_updated(xk):
#     global result
#     global presult
#     global count
#     global nit
#     global pmse
    
#     count+=1
#     nit+=1
#     if count==20:
#         count = 0
#         cmse = mse(xk,k, dvalid)
# #         print ccost
#         if cmse> pmse:
#             print 'here'
#             result[:] = presult
#             raise ConvergedException()
#             return
#         else:
#             pmse = cmse
#             presult[:] = xk

In [120]:
class ConvergedException(Exception):
    pass
def check_updated(xk):
    global result
    global minmse
    global count
    global presult
    global nit 
    
    nit +=1
#     print nit

#     cmse = mse(xk,k, dvalid)
    cmse = cost(xk, k, dvalid)
    if cmse > minmse:
            count +=1
            if count == 30:
                print 'here'
                result[:] = presult
                raise ConvergedException()
                return

    else:
        minmse = cmse
        presult[:] = xk
        count = 0
#     print cmse, minmse, presult[0]
    return
    

In [57]:
# lams = [0.001, 0.01, 0.1, 1, 10, 15, 20, 30]
# 10 - 15 - 20
lams = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
for l in lams:    
    presult = []
    count = 0
    nit = 0
    minmse = 10000000000000
    result = []
    pms = []
    try:
        pms, c,d = fmin_l_bfgs_b(compute_cost, x0=initialg,args=(l,k, dtrain, ), disp = 0, callback = check_updated)
        pms = presult
    except ConvergedException:
        pms = result
    print 'converged after: ' + str(nit)
    print "lambda " + '"' + str(l) +  '": ' +  ' ' + str(cost(pms, k, dvalid)) + ' ' + str(cost(pms,k, dtest)) + ' ' +  str(mse(pms, k, dvalid)) + ' ' + str(mse(pms,k, dtest))

here
converged after: 43
lambda "0":  10526.9900706 10348.7349736 0.785068988781 0.771717745979
here
converged after: 43
lambda "1e-05":  10526.9899274 10348.7348128 0.785068978103 0.771717733991
here
converged after: 43
lambda "0.0001":  10526.9886389 10348.7333661 0.785068882013 0.771717626109
here
converged after: 43
lambda "0.001":  10526.9757671 10348.7189148 0.785067922075 0.771716548457
here
converged after: 43
lambda "0.01":  10526.8483462 10348.5759702 0.785058419435 0.771705888902
here
converged after: 43
lambda "0.1":  10525.7023464 10347.3017475 0.784972954463 0.771610868571
here
converged after: 43
lambda "1":  10526.320546 10349.2164607 0.785019057796 0.77175365106
here
converged after: 71
lambda "10":  10340.6304533 10120.8356042 0.771170889201 0.754723012991
here
converged after: 41
lambda "100":  10725.899799 10484.2171235 0.799903035203 0.781820814575
converged after: 10
lambda "1000":  12381.4029892 12003.2303621 0.923365127097 0.895095478158
converged after: 6
lambd

In [61]:
import matplotlib
from pylab import *
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
lams = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
x = lams
y = [0.785068988781, 0.785068978103 , 0.785068882013 , 0.785067922075 , 0.785058419435 , 0.784972954463 , 0.785019057796 , 0.771170889201,
     0.799903035203 , 0.923365127097 , 0.981976525246 , 0.991621561514 
    ]
plt.scatter(lams[7], y[7], color = 'black', s = 100, marker="o")
plt.semilogx(x, y, color = 'red')plt.xlabel('Lambda Value For Adjacent Genre Bias Terms')
plt.ylabel('mse on Validation Set')
plt.title("Baseline Model Regularization Parameter vs. mse")
plt.show()

In [48]:
# lams = [0.001, 0.01, 0.1, 1, 10, 15, 20, 30]
# 10 - 15 - 20
lams = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
for l in lams:    
    presult = []
    count = 0
    nit = 0
    minmse = 1000
    result = []
    pms = []
    try:
        pms, c,d = fmin_l_bfgs_b(compute_cost, x0=initialg,args=(l,k, dtrain, ), disp = 0, callback = check_updated)
        pms = presult
    except ConvergedException:
        pms = result
    print 'converged after: ' + str(nit)
    print "lambda " + '"' + str(l) +  '": ' +  ' ' + str(mse(pms, k, dvalid)) + ' ' + str(mse(pms,k, dtest))

 here
converged after: 46
lambda "0":  0.787161757164 0.773036950299
here
converged after: 46
lambda "1e-05":  0.787161743079 0.773036933497
here
converged after: 46
lambda "0.0001":  0.78716161632 0.773036782289
here
converged after: 46
lambda "0.001":  0.78716034963 0.77303527105
here
converged after: 46
lambda "0.01":  0.787147772819 0.773020243193
here
converged after: 46
lambda "0.1":  0.787030971509 0.772878390922
here
converged after: 46
lambda "1":  0.786776946029 0.772346745893
here
converged after: 63
lambda "10":  0.770920726875 0.753850801872
converged after: 38
lambda "100":  0.814037917404 0.792408926021


In [115]:
# lams = [0.001, 0.01, 0.1, 1, 10, 15, 20, 30]
# 10 - 15 - 20
# lams = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
lams = [0, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
# with open('Regression baseline gamma and beta terms 100 core more.csv', 'wb') as f:
    writer = csv.DictWriter(f, fieldnames=['lambda','k','value'])
    writer.writeheader()
    for l in lams:    
        minmse = 100
        presult = []
        count = 0
        nit = 0
        result = []
        pms = []
        try:
            pms, c,d = fmin_l_bfgs_b(compute_cost, x0=initialg,args=(l,k, dtrain, ), disp = 0, callback = check)
        except ConvergedException:
            pms = result
        print 'converged'
        print "lambda " + '"' + str(l) +  '": ' +  ' ' + str(mse(pms, k, dtest))
        for i, v in enumerate(pms):
            writer.writerow({'lambda': l, 'k': k, 'value': v})  

here
converged
lambda "0":  0.767038174675
here
converged
lambda "0.0001":  0.767038093144
here
converged
lambda "0.001":  0.767037360122
here
converged
lambda "0.01":  0.76703010457
here
converged
lambda "0.1":  0.766964927139
here
converged
lambda "1":  0.76701374707
here
converged
lambda "10":  0.754400873307
converged
lambda "100":  0.792414488862
converged
lambda "1000":  0.900695509742


In [116]:
pms = defaultdict(lambda: defaultdict(list))
with open('Regression baseline gamma and beta terms 100 core more.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        pms[float(row['lambda'])][int(row['k'])].append(float(row['value']))

In [117]:
op = []
minCost = 1000000000
predictions = []
for l in pms:
    for k in pms[l]:
        c = mse(pms[l][k], k, dvalid)
#         co = cost(pms[l][k], 0 , 10, dvalid)
#         print l,  c, co
#         print l, c
        if c < minCost:
            minCost = c
            minlambda = l
            mink = k
            op = pms[l][k]
minlambda

10.0

In [125]:
# with open("Baseline Model Optimized Parameters.csv", "wb") as f:
#     writer = csv.writer(f)
#     for o in op:
#         writer.writerow([o])

In [126]:
op = []
with open("Baseline Model Optimized Parameters.csv", 'rb') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        op.append(float(row[0]))

In [127]:
print "mse: " + str(mse(op,k,dtest))
print "R Squared Statistic: " + str( mse(op, k, dtest) /float(variance(data)))

mse: 0.754400873307
R Squared Statistic: 0.767963744883


In [128]:
mse(op, k, dtrain)

0.55945122135694281

In [303]:
params = op
alphao = params[0]
betauo = params[1: len(users)+1]
end = len(users)+1
betaio = params[end: end+len(products)]
end = end+len(products)
gs = np.array(params[end:])
gr = gs.reshape(len(users)+len(products),k)
gammuo = gr[:len(users),:]
gammio = gr[len(users):,:]

In [305]:
product_names = defaultdict(str)
with open('100 core subset filtered.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        product_names[row['pid']] = row['ptitle']

In [192]:
user_ratings_with_time = defaultdict(lambda: list())
user_ratings = defaultdict(lambda: list())
for d in data:
    user_ratings_with_time[d[0]].append((d[1], int(d[3])))

for u in user_ratings_with_time:
    items = user_ratings_with_time[u]
    isorted = sorted(items, key = lambda tup: tup[1])
    user_ratings_with_time[u] = isorted
    user_ratings[u] = [i[0] for i in isorted]

In [193]:
import time
from datetime import datetime
print data[0][3]
def getTime(unix):
    time = datetime.utcfromtimestamp(float(unix))
    return int(time.strftime('%d')), int(time.strftime('%m')), int(time.strftime('%Y'))

1311984000


In [194]:
# time.strptime("30 Nov 00", "%d %b %y") 
# s = '30/11/2000'
# d,m,y = getTime(data[0][3])
# print d,m,y

In [195]:
#This user read harry potter at this unix time
user = 'ABMX8XUNPR3LP'
time = '1299888000'

In [196]:
#returns both items user has rated but not yet at this point in time and items not rated at all 
def get_products_not_rated(user, time):
    items = user_ratings_with_time[user]
    not_rated_yet = []
    for i,p in enumerate(items):
        if p[1] > int(time) and i > 0:
            not_rated_yet.extend([j[0] for j in items[i:]])
            break
    return ps.difference(user_ratings[user]).union(set(not_rated_yet))

In [297]:
def get_items_rated_in_future(user, time, pnames):
    items = user_ratings_with_time[user]
    not_rated_yet = []
    for i,p in enumerate(items):
        if p[1] > int(time) and i > 0:
            not_rated_yet.extend([j[0] for j in items[i:]])
            break
    names = []
    for n in not_rated_yet:
        names.append(pnames[n])
    return names

In [299]:
import operator
def getTopNRecommendations(user,time, n, pnames, params, mink):
    alphao = params[0]
    betauo = params[1: len(users)+1]
    end = len(users)+1
    betaio = params[end: end+len(products)]
    end = end+len(products)
    gs = np.array(params[end:])
    gr = gs.reshape(len(users)+len(products),k)
    gammuo = gr[:len(users),:]
    gammio = gr[len(users):,:]
    rankings = defaultdict(float)
    inr = get_products_not_rated(user,time)
    for r in inr:
        pred = betaio[products[r]] + (gammuo[users[user]].dot(gammio[products[r]]))
        rankings[r] = pred
    rs = sorted(rankings.iteritems(), key = operator.itemgetter(1), reverse= True)[:n]
    names = []
    for r in rs:
        names.append(pnames[r[0]])
    return names
    

In [300]:
with open('categories.txt', 'rb') as content_file:
    categories = content_file.read().split('\n')
#to make things faster i put the index where every category starts in a dictionary where the key are the product ids and 
#the values are the starting index of the category
catindex = defaultdict(int)
f = open('categories.txt','rb')
for i,l in enumerate(f):
    if len(l)-len(l.strip()) <=1:
        catindex[l.strip()]=i
        
#takes product id and returns category
def getCategory(pid, categories, catdict, depth):
    index = catdict[pid]
    value = set()
    j = index + 1
    while len(categories[j])-len(categories[j].strip())>1:
                ws = (categories[j].strip()).split(',')
                break
    return str((ws[:depth])[-1].strip())

In [301]:
import operator
def getTopNRecommendations_genres(user,time, n, params, k, pnames):
    alphao = params[0]
    betauo = params[1: len(users)+1]
    end = len(users)+1
    betaio = params[end: end+len(products)]
    end = end+len(products)
    gs = np.array(params[end:])
    gr = gs.reshape(len(users)+len(products),k)
    gammuo = gr[:len(users),:]
    gammio = gr[len(users):,:]
    rankings = defaultdict(float)
    inr = get_products_not_rated(user,time)
    for r in inr:
        pred = betaio[products[r]] + (gammuo[users[user]].dot(gammio[products[r]]))
        rankings[r] = pred
    rs = sorted(rankings.iteritems(), key = operator.itemgetter(1), reverse= True)[:n]
    cats = []
    for r in rs:
        cats.append(pnames[r[0]])
    return cats
    

In [315]:
recs = getTopNRecommendations_genres(user,time, 200, params, k, product_names)
# for r in recs:
#     print r, '\n'

In [316]:
count = 0
nyr = get_items_rated_in_future(user, time, product_names)
for r in recs:
    if r in nyr:
        print r
        count+=1
if count == len(nyr):
    print True

World War Z: An Oral History of the Zombie War
The time machine,: An invention,
The Giver
Lover Awakened (Black Dagger Brotherhood #3)
The Thirteenth Tale: A Novel
Charlotte's Web
Fables 1 (Turtleback School & Library Binding Edition)
Harry Potter & the Order of the Phoenix


In [317]:
import hashlib
def gethexdig(txt):
    c = hashlib.md5(txt.encode())
    return c.hexdigest()

In [318]:
gs= defaultdict(int)
for d in data:
    cat = getCategory(d[1], categories, catindex, 3)
    gs[cat]+=1
gs = set(sorted(gs.iteritems(), key= operator.itemgetter(1), reverse=True)[:50])
ghashed = set()
for g in gs:
    ghashed.add(gethexdig(g[0])) 
dg = 'anything'

In [324]:
pgi = defaultdict(float)
for p in products:
    cat = getCategory(p, categories,catindex,3)
#     if gethexdig(cat) not in ghashed:
#         cat = dg
#     pgi[(cat, product_names[p], p)] = gammio[products[p]][0]
    pgi[(p, product_names[p])] = gammio[products[p]][0]

In [325]:
pgiSorted = sorted(pgi.iteritems(), key = operator.itemgetter(1), reverse=True)

In [327]:
pgiSorted

[(('B0008BPKRO', "The lays of ancient Rome (Heath's English classics)"),
  0.21285415480379591),
 (('0345350804',
   'The Best of H. P. Lovecraft: Bloodcurdling Tales of Horror and the Macabre'),
  0.2052926061332872),
 (('0634002066', 'How to Play from a Fake Book (Keyboard Edition)'),
  0.20508789820939613),
 (('0613925955',
   'Blankets: An Illustrated Novel (Turtleback School & Library Binding Edition)'),
  0.20462071253645195),
 (('B0006E4NRG', 'The captive mind'), 0.20351024920020075),
 (('B000GRO0WQ', 'The Outsider'), 0.20067364503111729),
 (('0131523597',
   'Writing Academic English, Fourth Edition (The Longman Academic Writing Series, Level 4)'),
  0.19934957676726053),
 (('B000IOF23I', 'Misfit:: The Strange Life of Frederick Exley'),
  0.19865635498593798),
 (('B0006C3SV0', 'Europe leaves the Middle East, 1936-1954'),
  0.19857548059601757),
 (('0385720866',
   'Lost Classics: Writers on Books Loved and Lost, Overlooked, Under-read, Unavailable, Stolen, Extinct, or Otherwise